In [1]:
from bertviz import attention, visualization
from bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer
import numpy as np
import h5py
import json 

bert_version = 'bert-large-uncased'
model = BertModel.from_pretrained(bert_version)
tokenizer = BertTokenizer.from_pretrained(bert_version)


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
filename = '/misc/vlgscratch4/BowmanGroup/datasets/ptb_trees/ptb3-wsj-test.json'
with open(filename, 'r') as f_wsj:
    data = json.load(f_wsj)

In [7]:
data[list(data.keys())[0]]

{'sentence': "No , it was n't Black Monday .",
 'dependencies': [[[1, 7], 'discourse'],
  [[2, 7], 'punct'],
  [[3, 7], 'nsubj'],
  [[4, 7], 'cop'],
  [[5, 7], 'neg'],
  [[6, 7], 'nn'],
  [[7, 0], 'root'],
  [[8, 7], 'punct']],
 'tokens': ['No', ',', 'it', 'was', "n't", 'Black', 'Monday', '.']}

In [10]:
attention={}
for key, datum in data.items():
    sent = datum['sentence']
    tokens = tokenizer.tokenize(sent)
    tokens_a_delim = ['[CLS]'] + tokens + ['[SEP]']
    token_ids =  tokenizer.convert_tokens_to_ids(tokens_a_delim)
    tokens_tensor = torch.tensor([token_ids])
    token_type_tensor = torch.LongTensor([[0] * len(tokens_a_delim)])
    _, _, attn_data_list =  model(tokens_tensor, token_type_ids=token_type_tensor)
    attn_tensor = torch.stack([attn_data['attn_probs'] for attn_data in attn_data_list])
    attention[key] = attn_tensor.data.numpy()

{'sentence': "No , it was n't Black Monday .", 'dependencies': [[[1, 7], 'discourse'], [[2, 7], 'punct'], [[3, 7], 'nsubj'], [[4, 7], 'cop'], [[5, 7], 'neg'], [[6, 7], 'nn'], [[7, 0], 'root'], [[8, 7], 'punct']], 'tokens': ['No', ',', 'it', 'was', "n't", 'Black', 'Monday', '.']}
{'sentence': "No , it was n't Black Monday .", 'dependencies': [[[1, 7], 'discourse'], [[2, 7], 'punct'], [[3, 7], 'nsubj'], [[4, 7], 'cop'], [[5, 7], 'neg'], [[6, 7], 'nn'], [[7, 0], 'root'], [[8, 7], 'punct']], 'tokens': ['No', ',', 'it', 'was', "n't", 'Black', 'Monday', '.']}


In [12]:
L=len(data)
print("writing weights to the file!!")
with h5py.File('/misc/vlgscratch4/BowmanGroup/datasets/ptb_trees/bert_large_wsj_test.hdf5','w') as f:
    for idx in attention:
        f.create_dataset(idx,data=attention[idx],dtype='float64')
f.close()
print("done") 

2416